In [42]:
sourcePathName="/Users/sertoez/git-projects/period-suite/"

import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-200;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
periodsOnly=true; #unless overridden 
paths=[]; #unless overridden we will convert this to a list of straight paths

#load("~/git-projects/periods/integrate/data/current.sage")
t0=time.time();
load(sourcePathName+"current.sage")
t1=time.time();
print "Loaded file in", t1-t0, "seconds";
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "\nIntegrating system number", j;
    transitionMatricies = []
    for i in [1..noOfdeqs]:
        tt0=time.time()
        tm=deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true)
        print "\tODE number", i, "took", time.time()-tt0, "seconds to integrate"
        transitionMatricies=transitionMatricies+[tm]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:noOfdeqs]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods



# Write the periods to file

def row2string(row):
    strRow=str(row[0])
    for i in [1..(len(row)-1)]:
        strRow=strRow+", "+str(row[i])
    return "["+strRow+"]"
    
outputFile = open(sourcePathName+"lastPeriods",'w')

outputFile.write("M:=Matrix(["+row2string(periods[0]))
for i in [1..(periods.nrows()-1)]:
    outputFile.write(",\n"+row2string(periods[i]))
outputFile.write("]);")
outputFile.close()


Loaded file in 0.889204978943 seconds
Integrating systems 1 through 5

Integrating system number 1
	ODE number 1 took 4.59705781937 seconds to integrate
	ODE number 2 took 4.25999498367 seconds to integrate
	ODE number 3 took 4.0173330307 seconds to integrate
	ODE number 4 took 3.2987370491 seconds to integrate
	ODE number 5 took 6.26508998871 seconds to integrate
	ODE number 6 took 4.7669210434 seconds to integrate
	ODE number 7 took 3.61520290375 seconds to integrate
	ODE number 8 took 4.30941796303 seconds to integrate
	ODE number 9 took 3.33620715141 seconds to integrate
	ODE number 10 took 5.97635197639 seconds to integrate
	ODE number 11 took 4.85194301605 seconds to integrate
	ODE number 12 took 3.71766090393 seconds to integrate
	ODE number 13 took 4.04539704323 seconds to integrate
	ODE number 14 took 4.62722396851 seconds to integrate
	ODE number 15 took 6.28357505798 seconds to integrate
	ODE number 16 took 4.92075300217 seconds to integrate
	ODE number 17 took 3.61498403549

In [54]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 60
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 80
##
##

load("IntersectionMatrix.sage")

periodVector=periods*intmat.inverse()
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms2

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"
print "First row records the norm of the relation."
print "Second row shows if the relation continues to hold with more digits.\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms1[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms1[j], testRelations[j]
    j=j+1



Picard rank appears to be 4 

Check to see if this makes sense:

First row records the norm of the relation.
Second row shows if the relation continues to hold with more digits.

01 || ---hyperplane section---
02 || 2.00000000000000 (0, 0)
03 || 2.00000000000000 (0, 0)
04 || 2.00000000000000 (0, 0)
05 || 9.66856197779013e6 (526841687463743020141394555, 156126993400013589110361623)
06 || 9.55999713244701e6 (146657935261844629256021221, 132025405593863142673555653)
07 || 1.01110351503484e7 (-361543221655119405747192374, 147351307256732747858866541)
08 || 9.69856161560435e6 (89359670865523808704570883, -59165565684417548429088171)
09 || 1.01224101534816e7 (-435829375145510078497019720, -351528576706741188571360205)
10 || 9.89998970930864e6 (-16665114610463270119505652, 486982779135497488954005626)
11 || 1.02246468360071e7 (-688063587701081202041367354, 105660220945004779461688813)
12 || 1.03245163882842e7 (-136138743141985532325798089, 268353603282671100838553976)
13 || 1.13222621402430e